# Bases des télécommunications 1 : Codage de Huffman. Costantino Volta, Jeremy Meissner.

## __Introduction__

Le but de ce laboratoire est de mettre en pratique les concepts de compression de données vus en classe de Bases des
Télécommunications. On travaillera sur la compression d’un fichier texte

- Le rendu du **jupyter notebook** doit se faire sur __cyberlearn__ avec le nom de fichier : `telecom_labo1_<nom1>_<nom2>_<nom du groupe>.ipynb`
- Le fichier texte utilisé ici doit être encodé en UTF-8 (sans BOM), vous pouvez le vérifier avec Notepad++ par exemple.
- Exemple de code python 3 pour ouvrir en lecture un fichier texte en UTF-8 : __*open(INPUT, "r", encoding="UTF-8")*__


Noms des élèves : `Jeremy Meissner - Costantino Volta`

<h2>I. Tris alphabétique + fréquentiel</h2>

a)  Réalisez  ci-dessous un programme qui lit un fichier texte (input.txt), puis en écrit un nouveau (out1.txt)
en écrivant tous les caractères dans l’ordre alphabétique (ordre de leur valeur
ASCII).
Exemple : **abb@acA → @Aaabbc** (La fonction `sorted` de Python 3 pourrait vous aider, tout comme l’utilisation
de dictionnaires)

In [35]:
import math

# exercice a.
sortedInputList = []
inputList = []
with open("input.txt", "r") as file:
    contents = file.read()
    inputList = list(contents) # Convert the file into an array of char

sortedInputList = sorted(inputList) 

with open("out1.txt", "w") as file:
    for c in sortedInputList:
        file.write(c)

b) Ajoutez à votre premier programme la possibilité d'écrire dans un nouveau fichier texte
(out2.txt), mais cette fois, les données sont triés dans l’**ordre des fréquences d’apparition décroissante**.

``Exemple : abb@acA → aabb@Ac``

Votre programme doit afficher dans la console, la fréquence d’apparition pour
chaque caractère. (Ainsi que la valeur ascii, hexadécimal et la fréquence d'apparition du caractère)

Exemple : 

<table>
<tr>
    <td>a</td>
    <td>0x61</td>
    <td>0.285</td>
</tr>
<tr>
    <td>b</td>
    <td>0x62</td>
    <td>0.285</td>
</tr>
<tr>
    <td>A</td>
    <td>0x41</td>
    <td>0.142</td>
</tr>
</table>



In [36]:
# exercice b.
sortedByFrequence = {}

for c in inputList:
    sortedByFrequence[c] = sortedByFrequence.get(c, 0) + 1

 # sortedByFrequence.sortedByItsValue()???? <--- A dictionnary CAN'T be sort by his values


sortedByFrequence = sorted(sortedByFrequence.items(), key=lambda x: x[1], reverse=True) # So using a lambda (that will create tuples and order by the second value in the tuple (which is the value))
# was {'L': 3, 't': 34, 'g': 2, ...} <- Dictionnary
# now [(' ', 61), ('e', 49), ('t', 34), ...] <- List of Tuple

with open("out2.txt", "w") as file:
    for (char, frequency) in sortedByFrequence:
        for _ in range(0, frequency):
            file.write(char)



sumFrequencies = sum(item[1] for item in sortedByFrequence)

print(f"char | ascii | frequency")
for (char, frequency) in sortedByFrequence:
    print(f"{char} | {hex(ord(char))} | {round(frequency / sumFrequencies, 3)}")

char | ascii | frequency
  | 0x20 | 0.151
e | 0x65 | 0.122
t | 0x74 | 0.084
u | 0x75 | 0.062
s | 0x73 | 0.06
a | 0x61 | 0.057
n | 0x6e | 0.057
r | 0x72 | 0.05
i | 0x69 | 0.047
o | 0x6f | 0.042
l | 0x6c | 0.037
d | 0x64 | 0.03
m | 0x6d | 0.03
p | 0x70 | 0.022
c | 0x63 | 0.017
. | 0x2e | 0.017
' | 0x27 | 0.012
x | 0x78 | 0.012
é | 0xe9 | 0.012
v | 0x76 | 0.01
L | 0x4c | 0.007
I | 0x49 | 0.007
q | 0x71 | 0.007
D | 0x44 | 0.007
b | 0x62 | 0.007

 | 0xa | 0.007
g | 0x67 | 0.005
C | 0x43 | 0.005
è | 0xe8 | 0.002
f | 0x66 | 0.002
ç | 0xe7 | 0.002
à | 0xe0 | 0.002
, | 0x2c | 0.002


c) Améliorez votre programme pour qu’il calcule l’entropie, la quantité de décision,
la redondance et le taux de compression maximal du fichier. Le tout est à afficher
dans la console.

**⚠️ N’oubliez pas que les espaces, la ponctuation et le retour chariot sont aussi considéré comme des
caractères. 
Votre programme doit pouvoir traiter ces caractères de manière adéquate.**


In [37]:
# exercice c.
entropyH = 0
entropyMaxD = 0
redondancyR = 0
compressionRatio = 0


for (char, frequency) in sortedByFrequence:
    entropyH += frequency / sumFrequencies * math.log2(frequency / sumFrequencies)

entropyH = entropyH * -1 # Because the log makes the entropy negative
entropyMaxD = math.log2(len(sortedByFrequence))
redondancyR = entropyMaxD - entropyH
compressionRatio = (entropyMaxD - entropyH) / entropyMaxD


print(f"H = {entropyH}")
print(f"D = {entropyMaxD}")
print("")
print(f"R = D - H")
print(f"{round(redondancyR, 2)} = {round(entropyMaxD, 2)} - {round(entropyH, 2)}")
print("")
print(f"R = {redondancyR}")
print("")
print(f"Compression Rate = {round(compressionRatio, 2)}")

H = 4.266154878255668
D = 5.044394119358453

R = D - H
0.78 = 5.04 - 4.27

R = 0.778239241102785

Compression Rate = 0.15


<h2>II. Code de Huffman</h2>

On veut construire un code de Huffman pour compresser notre fichier texte. On part du principe
que le code n’a pas besoin d’être transmis dans le fichier compressé. 


a) À partir de l’exercice précédent (utilisez les mêmes valeurs précement calculées), écrivez un code générant un codage de Huffman pour
les caractères de ce fichier. Votre programme doit imprimer l'arbre de codage que vous avez générer. Affichez la valeur ASCII des
charactères ainsi que leur code binaire généré par codage de Huffman.

Exemple:

<table>
<tr>
    <td>0x61</td>
    <td>11</td>
</tr>
<tr>
    <td>0x62</td>
    <td>10</td>
</tr>
<tr>
    <td>0x40</td>
    <td>00</td>
</tr>
<tr>
    <td>0x63</td>
    <td>011</td>
</tr>
<tr>
    <td>0x41</td>
    <td>010</td>
</tr>
</table>

In [38]:
# exercice a.
class Node:
   def __init__(self, data):
      self.left = None
      self.right = None
      self.data = data

   def ArrayHuffman(self, binaryCode, array):
      if isinstance(self.data, tuple):
         array.append((self.data[0], hex(ord(self.data[0])), binaryCode))
      else:
         if self.left:
            self.left.ArrayHuffman(binaryCode + "0", array)
         if self.right:
            self.right.ArrayHuffman(binaryCode + "1", array)


huffmanReady = [(char, frequency / sumFrequencies) for char, frequency in sortedByFrequence]


for i in range(len(huffmanReady)-1,0, -1):
    node2data = 0
    if not isinstance(huffmanReady[i][0], Node):
        node0 = Node(huffmanReady[i])
        node2data += node0.data[1]
    else:
        node0 = huffmanReady[i][0] 
        node2data += node0.data
    
    if not isinstance(huffmanReady[i-1][0], Node):
        node1 = Node(huffmanReady[i-1])
        node2data += node1.data[1]
    else:
        node1 = huffmanReady[i-1][0] 
        node2data += node1.data

    node2 = Node(node2data)
    node2.left = node0
    node2.right = node1
    huffmanReady.pop(i)
    huffmanReady.pop(i-1)
    huffmanReady.append((node2, node2.data))
    huffmanReady = sorted(huffmanReady, key=lambda x: x[1], reverse=True)


huffmanArray = []
root = huffmanReady[0][0]
root.ArrayHuffman("", huffmanArray)
huffmanArray = sorted(huffmanArray, key=lambda x: len(x[2]))

for s in huffmanArray:
   print(f"{s[1]} | {s[2]}")

0x65 | 100
0x20 | 110
0x69 | 0001
0x72 | 0010
0x6e | 0100
0x61 | 0101
0x73 | 0111
0x75 | 1010
0x74 | 1111
0x70 | 00000
0x6d | 01100
0x64 | 01101
0x6c | 10111
0x6f | 11101
0xe9 | 000011
0x78 | 001100
0x27 | 001101
0x2e | 101101
0x63 | 111000
0x67 | 0000100
0x49 | 0011100
0x4c | 0011101
0x44 | 0011110
0x71 | 0011111
0xa | 1011000
0x62 | 1011001
0x76 | 1110011
0xe8 | 00001010
0x43 | 11100101
0xe7 | 000010110
0x66 | 000010111
0x2c | 111001000
0xe0 | 111001001


b) Afficher le taux moyen de bits par caractère de votre code, dans la console.

In [39]:
summ = 0

for s in huffmanArray:
   summ += len(s[2])

lm = summ / len(huffmanArray)
print("lm is", lm)

lm is 5.96969696969697


 c) Que pouvez-vous conclure du résultat obtenu à l'exercice b) et de l’entropie du point (I.c), en une phrase ?


Notre lm est de 5.96....
L'entropy est de 4.555

On peut directement remarquer que le codage Huffman est sous-optimal.

On peut également calculer la redondance résiduel ci-dessous (qui est d'environ 1.7). On peut donc conclure qu'il est possible de encore possible de trouver un codage plus optimal.

In [40]:
Rr = lm - entropyH

print(f"Rr = lm - H = {round(lm, 2)} - {round(entropyH, 2)} = {round(Rr, 2)}")

Rr = lm - H = 5.97 - 4.27 = 1.7


d) Compressez le fichier texte d’input dans un nouveau fichier binaire (huffman.bin) à l'aide de votre arbre de codage. Si la longueur du message codé n’est pas un multiple de 8, rajoutez des bits 0 à la fin. Le
fichier compressé doit avoir une taille sur disque significativement plus petite que
celui que le fichier source.

In [41]:
outputList = []
summ = 0

for c in inputList:
    for huf in huffmanArray:
        if huf[0] == c:
            outputList.append(huf[2])
            summ += len(huf[2])
            break

numberOfAddedZero = 0 if (summ % 8) == 0 else 8 - (summ % 8) 

outputString = ""
for s in outputList:
    outputString += s

outputString += "0" * numberOfAddedZero

byteLength = (len(outputString) + 7) // 8


with open("huffman.bin", "wb") as file:
    file.write(int(outputString, 2).to_bytes(byteLength, byteorder="big"))

e) Que manquerait-il à ce fichier binaire pour être décodé par n’importe qui ?
(Bonus)

Il faudrait mettre dans l'entête le dictionnaire du code huffman qui permettrait à la personne qui reçoi de pouvoir décoder les charactères (à partir des codes binaires reçu)

# Bonus

Codez par vous même la fonction de tri sans faire appel à la fonction `sorted()` de python

In [42]:
tabOfChars = ['b', 'a', 'a', 'f']# transformation du fichier lu en tableau de charactères

def mySortedFunc(tab):
    def find_idx_min(tab):
        min_val = tab[0]
        idx_min = 0
        for i in range(len(tab)):
            if tab[i] < min_val:
                min_val = tab[i]
                idx_min = i
        return idx_min

    def swap(tab, idx_x, idx_y):
        tab[idx_x], tab[idx_y] = tab[idx_y], tab[idx_x]

    def selection_sort(tab):
        for i in range(len(tab)):
            idx_min = find_idx_min(tab[i:])
            swap(tab, i, i + idx_min)

    selection_sort(tab)
    return tab

sortedChars = mySortedFunc(tabOfChars)
print(sortedChars)

['a', 'a', 'b', 'f']


In [45]:
from IPython.display import display, HTML, Image

resolved = sorted(tabOfChars) == mySortedFunc(tabOfChars)
    
# Validation
js_code = "var idx = Jupyter.notebook.get_cells().length-1;\n"
if resolved:
    js_code += "Jupyter.notebook.get_cell(idx).set_text('Bravo! ![success](https://media.tenor.com/-8Uay6X3E3UAAAAC/gil-cat.gif)');\n"
else:
    js_code += "Jupyter.notebook.get_cell(idx).set_text('Incorrect! ![failed](https://media.tenor.com/jr9t3yabkH8AAAAC/ah-shit-here-we-go-again.gif)');\n"

js_code += "Jupyter.notebook.to_markdown(idx);\nJupyter.notebook.execute_cell(idx);"

display(HTML('<script>{}</script>'.format(js_code)))


Fin